In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.prompt import PromptTemplate
from langchain.memory import ConversationSummaryBufferMemory
from langchain.schema.runnable import RunnablePassthrough

# 이 체인은 영화 제목을 가져와 영화를 나타내는 세 개의 이모티콘으로 응답해야 합니다. (예: "탑건" -> "🛩️👨‍✈️🔥". "대부" -> "👨‍👨‍👦🔫🍝").
# 항상 세 개의 이모티콘으로 답장하도록 FewShotPromptTemplate 또는 FewShotChatMessagePromptTemplate을 사용하여 체인에 예시를 제공하세요.
# 메모리가 작동하는지 확인하려면 체인에 두 개의 영화에 대해 질문한 다음 다른 셀에서 체인에 먼저 질문한 영화가 무엇인지 알려달라고 요청하세요.
llm = ChatOpenAI(temperature=0.1)

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler()]
)

# 과거 답변 내용을 기억
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=80,
    memory_key="chat_history",
    return_messages=True
)

examples = [{
    "question": "What do you know about Harry Potter series? Reply with three emojis only",
    "answer": "🧙🏻‍♀️🪄🏫",
    "question": "What do you know about God Father? Reply with three emojis only",
    "answer": "🔫🇮🇹🇺🇸",
    }]

example_prompt = PromptTemplate.from_template("Human: {question}¥nAI:{answer}")

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    suffix="{question}?",
    input_variables=["question"],
    )

def load_memory(_):
    return memory.load_memory_variables({})["chat_history"]

chain = RunnablePassthrough.assign(chat_history=load_memory) | prompt | llm

def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context({"input":question}, {"output": result.content})
    print(result)

In [3]:
invoke_chain(question="Do you know about Spiderman?")
invoke_chain(question="Do you know about Notting Hill?")

content='🕷️🕸️🦸\u200d♂️'
content='🎬🏴📽️'


In [4]:
invoke_chain(question="Do you remember your answer about Spiderman?")

content="I'm sorry, I don't have the ability to remember previous interactions. How can I assist you with Spiderman?"
